In [1]:
from adam.casadi.computations import KinDynComputations
from adam.geometry import utils
import numpy as np
import casadi as cs
import tempfile
from git import Repo

## Import the robot .urdf

In [2]:
# Getting stickbot urdf file
temp_dir = tempfile.TemporaryDirectory()
git_url = "https://github.com/icub-tech-iit/ergocub-gazebo-simulations.git"
Repo.clone_from(git_url, temp_dir.name)
model_path = temp_dir.name + "/models/stickBot/model.urdf"
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
# Specify the root link
root_link = 'root_link'
link_parametric_list = ["chest"]
kinDyn = KinDynComputations(
    model_path, joints_name_list, root_link, link_parametric_list
)

Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']
Unknown tag "sensor" in /robot[@name='stickBot']


The class `KinDynComputations` contains some algorithms for computing kinematics and dynamics quantities.

In [3]:
# Set joints and base informations
xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_Pos_RPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
lenght_multiplier = np.random.rand(kinDyn.urdf_tree.NLinkParametric,3) # it is a matrix with 3 cols, to elongate in all the direction any shape in [sphere, cylinder, box]
density = np.random.rand(kinDyn.urdf_tree.NLinkParametric)

## Mass Matrix

In [4]:
M = kinDyn.mass_matrix_fun()
print('Mass matrix:\n', cs.DM(M(H_b, s, density, lenght_multiplier)))

Mass matrix:
 
[[46.4992, 2.13163e-14, 4.9738e-14, 6.88338e-15, -1.53415, -0.326347, 0.818163, 1.3762, 0.771508, -0.244036, -0.454816, -0.130094, 0.0476307, 0.206132, 0.389343, 0.095125, 0.0235011, -0.468203, 1.47275, 0.0684183, -1.00748, -0.00522387, 0.0280196, -0.720628, 0.912299, 0.580887, -0.414383, 0.0495706, -0.0302817], 
 [1.77636e-14, 46.4992, 1.42109e-14, 1.53415, -5.70377e-15, 2.1897, -3.89377, 0.202411, 0.283686, 0.031513, -0.22444, -0.0384925, -0.144292, -0.263545, 0.217043, 0.00567683, -0.143991, 0.508996, 1.91219, 0.986506, 0.0175226, -0.00232661, -0.021921, -0.184965, 0.923177, 0.278969, 1.10865, -0.0714158, -0.013488], 
 [4.9738e-14, 5.32907e-15, 46.4992, 0.326347, -2.1897, -1.4988e-15, 0.57676, 0.0641575, -0.00811097, 0.434752, -0.184832, 0.0302001, 0.021793, 0.302332, -0.0585416, 0.0281789, -0.0468693, 0.906501, -1.95957, -0.018095, 0.394425, 0.0345722, -0.0107607, 0.729094, 1.10493, -0.439835, 0.0275586, 0.0250963, 0.0168358], 
 [6.55032e-15, 1.53415, 0.326347, 2.178

## Centroidal Momentum Matrix

In [5]:
Jcm = kinDyn.centroidal_momentum_matrix_fun()
print('Centroidal Momentum Matrix:\n', cs.DM(Jcm(H_b, s, density, lenght_multiplier)))

Centroidal Momentum Matrix:
 
[[46.4992, 2.13163e-14, 4.9738e-14, 6.88338e-15, -1.53415, -0.326347, 0.818163, 1.3762, 0.771508, -0.244036, -0.454816, -0.130094, 0.0476307, 0.206132, 0.389343, 0.095125, 0.0235011, -0.468203, 1.47275, 0.0684183, -1.00748, -0.00522387, 0.0280196, -0.720628, 0.912299, 0.580887, -0.414383, 0.0495706, -0.0302817], 
 [1.77636e-14, 46.4992, 1.42109e-14, 1.53415, -5.70377e-15, 2.1897, -3.89377, 0.202411, 0.283686, 0.031513, -0.22444, -0.0384925, -0.144292, -0.263545, 0.217043, 0.00567683, -0.143991, 0.508996, 1.91219, 0.986506, 0.0175226, -0.00232661, -0.021921, -0.184965, 0.923177, 0.278969, 1.10865, -0.0714158, -0.013488], 
 [4.9738e-14, 5.32907e-15, 46.4992, 0.326347, -2.1897, -1.4988e-15, 0.57676, 0.0641575, -0.00811097, 0.434752, -0.184832, 0.0302001, 0.021793, 0.302332, -0.0585416, 0.0281789, -0.0468693, 0.906501, -1.95957, -0.018095, 0.394425, 0.0345722, -0.0107607, 0.729094, 1.10493, -0.439835, 0.0275586, 0.0250963, 0.0168358], 
 [1.23932e-14, -6.21812e

## Center of mass position

In [6]:
CoM = kinDyn.CoM_position_fun()
print('Center of Mass position:\n', cs.DM(CoM(H_b, s, density, lenght_multiplier)))

Center of Mass position:
 [-0.23607, -1.86112, -2.30606]


## Total Mass

In [7]:
m = kinDyn.get_total_mass()
print('Total mass:\n', m(density, lenght_multiplier))

m:(i0,i1[1x3])->(o0) SXFunction
Total mass:
 46.4992


## Jacobian

In [8]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian of the left sole:\n', cs.DM(J(H_b, s, density, lenght_multiplier)))

Jacobian of the left sole:
 
[[1, 00, 00, 0, 0.274088, -0.054885, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.128168, 0.228036, 0.0244088, -0.221184, -0.0226372, 0.0496049, 0, 0, 0, 0, 0, 0], 
 [00, 1, 00, -0.274088, 0, 0.280313, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0148652, 0.247698, 0.214878, 0.00481842, 0.000735295, -0.0388082, 0, 0, 0, 0, 0, 0], 
 [00, 00, 1, 0.054885, -0.280313, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.142078, -0.247435, -0.0424664, 0.0735209, 0.00426639, -0.0190503, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 1, 00, 00, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.699579, -0.1116, -0.968963, -0.0463115, 0.0463115, -0.298902, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 00, 1, 00, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.40497, 0.751861, 0.062816, -0.99618, 0.99618, 0.0845409, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 00, 00, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.588718, 0.649808, -0.239093, -0.0740379, 0.0740379, -0.950532, 0, 0, 0, 0, 0, 0]]


## Relative jacobian

In [9]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian between the root link and left sole:\n', cs.DM(J_r(s, density, lenght_multiplier)))

Jacobian between the root link and left sole:
 
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.10179, -0.0157382, 0.125388, 0.0832994, 0.0065265, -0.0479815, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0864504, 0.094944, 0.113404, 0.0136226, -0.0392035, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.162705, -0.408482, -0.154386, 0.185881, 0.0174073, -0.0221472, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.999259, 0.0206454, -0.771501, 0.771501, -0.531523, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0.844063, -0.327437, 0.327437, 0.802937, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0384999, 0.535847, 0.545501, -0.545501, -0.269769, 0, 0, 0, 0, 0, 0]]


## Forward kinematics

In [10]:
H = kinDyn.forward_kinematics_fun('l_sole')
print('Forward kinematics to the left sole:\n', cs.DM(H(H_b, s, density, lenght_multiplier)))

Forward kinematics to the left sole:
 
[[-0.298902, 0.753874, -0.585091, -0.00284834], 
 [0.0845409, -0.58979, -0.803119, -1.81325], 
 [-0.950532, -0.289518, 0.112556, -1.99898], 
 [0, -0, 0, 1]]


## Bias force term

In [11]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', cs.DM(h(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Bias force term:
 [28.1918, -25.691, 494.501, -3.47499, -28.9125, -1.46761, 16.0623, 7.10326, -0.815852, 6.16206, -4.92271, 0.496181, -0.398394, 6.24579, -1.16804, 0.399526, -0.606732, 7.67881, -18.0411, -2.12994, 3.00602, 0.4264, -0.124693, 7.99205, 10.5934, -5.13802, -0.774876, 0.300273, 0.281313]


## Coriolis term

In [12]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', cs.DM(C(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Coriolis force term:
 [28.1918, -25.691, 38.4999, -6.67535, -7.43885, -1.46761, 10.4062, 6.47409, -0.736311, 1.8986, -3.11012, 0.200019, -0.612111, 3.28093, -0.593944, 0.123185, -0.147101, -1.21092, 1.17565, -1.95249, -0.861962, 0.0873626, -0.0191667, 0.842075, -0.242302, -0.824721, -1.04513, 0.0541629, 0.11621]


## Gravity term

In [13]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', cs.DM(G(H_b, s, density, lenght_multiplier)))

Gravity term:
 [4.54747e-13, 4.26326e-14, 456.001, 3.20037, -21.4736, -1.68754e-14, 5.65609, 0.62917, -0.0795414, 4.26346, -1.81259, 0.296162, 0.213717, 2.96486, -0.574097, 0.276341, -0.459631, 8.88974, -19.2168, -0.177451, 3.86798, 0.339037, -0.105526, 7.14997, 10.8357, -4.3133, 0.270258, 0.24611, 0.165102]
